In [27]:
import numpy as np
from qutip import *

# Parameters
phi = np.pi / 2  # Example phase on red link
sites = 4
links = [(0, 1), (1, 2), (1, 3)]  # Blue links
red_link = (1, 3)

# Basic operators
a = destroy(2)
adag = a.dag()
n = adag * a
I = qeye(2)

def embed(op, site, N):
    """Place op at site in N-qubit tensor product"""
    ops = [I]*N
    ops[site] = op
    return tensor(ops)

def two_site_term(i, j, phase, N):
    """Construct a†_i t a_j + h.c. term"""
    a_i_dag = embed(adag, i, N)
    a_j = embed(a, j, N)
    term = a_i_dag * a_j * phase + (a_i_dag * a_j * phase).dag()
    return term

# Build Hamiltonian
H = 0
for i, j in links:
    H += two_site_term(i, j, phase=1.0, N=sites)

# Red link with Aharonov-Bohm phase
i, j = red_link
H += two_site_term(i, j, phase=np.exp(1j * phi), N=sites)

V = 1.0
for i in range(sites):
    for j in range(i+1, sites):
        ni = embed(n, i, sites)
        nj = embed(n, j, sites)
        H += V * ni * nj

# Diagonalize
eigvals = H.eigenenergies()
print(np.round(np.sort(eigvals), 6))


[-2.       -1.288246 -0.414214 -0.        0.        0.        0.125968
  1.        1.874032  2.        2.414214  3.        3.        3.288246
  5.        6.      ]


In [16]:
import pennylane as qml
from pennylane.fermi import FermiC, FermiA, jordan_wigner
import networkx as nx
import numpy as np

# Parameters
phi = np.pi / 2 #0.5  # Aharonov-Bohm phase
eta = np.exp(1j * phi)  # Phase factor
N = 4  # Number of sites
edges = [(1, 2), (2, 3), (2, 4), (1, 3), (3, 4)]
red_link = (2, 4)  # Phase is only on this edge

# Build connectivity graph
G = nx.Graph()
G.add_edges_from(edges)

# Define path function between qubits
def jw_path(i, j):
    path = nx.shortest_path(G, i, j)
    return path[1:-1] if len(path) > 2 else []

# Get unique i<j with paths
nodes = sorted(set(i for e in edges for i in e))
triples = [(i, j, jw_path(i, j)) for i in nodes for j in nodes if i < j]

# Anyonic Sigma dagger operator (fermionic version)
# def sigma_dagger(k, theta):
#     a_k_dag = FermiC(k)
#     a_k = FermiA(k)
#     n_k = a_k_dag * a_k
#     id_op = qml.fermi.FermiSentence({})
#     z_k = id_op - 2 * n_k
#     #term_I = 0.5 * np.exp(1 + 1j * theta) * id_op
#     #term_Z = 0.5 * np.exp(-1 + 1j * theta) * z_k
#     term_I = 0.5 * np.exp(1j * theta) * 1
#     term_Z = 0.5 * np.exp(1j * theta) * Z
#     return term_I + term_Z

def sigma_dagger(k, theta):
    id_op = qml.fermi.FermiSentence({})
    n_k = FermiC(k) * FermiA(k)
    z_k = id_op - 2 * n_k
    return 0.5 * (np.exp(1j * theta) + 1) * id_op + 0.5 * (np.exp(1j * theta) - 1) * z_k

# Build Hamiltonian using Jordan-Wigner transformation
coeffs = []
ops = []

for (i, j, path) in triples:
    i_idx = i - 1
    j_idx = j - 1
    a_i_dag = FermiC(i_idx)
    a_j = FermiA(j_idx)

    phase = eta if (i, j) == red_link or (j, i) == red_link else 1.0

    # Build Sigma dagger product along path
    sigmas = [sigma_dagger(k - 1, phi) for k in reversed(path)]

    f_op = a_i_dag
    for s in sigmas:
        f_op = f_op * s
    f_op = f_op * a_j
    f_op *= phase

    qubit_op = jordan_wigner(f_op)
    ops.extend([qubit_op, qubit_op.adjoint()])
    coeffs.extend([1.0, 1.0])

# Add interaction terms: n_i n_j
V = 1.0  # Interaction strength
for i in range(N):
    for j in range(i + 1, N):
        ni = FermiC(i) * FermiA(i)
        nj = FermiC(j) * FermiA(j)
        f_interaction = ni * nj
        qubit_op = jordan_wigner(f_interaction)
        ops.append(qubit_op)
        coeffs.append(V)

# Hamiltonian
H = qml.Hamiltonian(coeffs, ops)

# Iterate over each term in the Hamiltonian and print the Pauli string
for i, op in enumerate(H.ops):
    print(f"Pauli string of term {i+1}: {op}")


Pauli string of term 1: -0.25j * (Y(0) @ X(1)) + (0.25+0j) * (Y(0) @ Y(1)) + (0.25+0j) * (X(0) @ X(1)) + 0.25j * (X(0) @ Y(1))
Pauli string of term 2: Adjoint(-0.25j * (Y(0) @ X(1))) + Adjoint((0.25+0j) * (Y(0) @ Y(1))) + Adjoint((0.25+0j) * (X(0) @ X(1))) + Adjoint(0.25j * (X(0) @ Y(1)))
Pauli string of term 3: -0.25j * (Y(0) @ Z(1) @ X(2)) + (0.25+0j) * (Y(0) @ Z(1) @ Y(2)) + (0.25+0j) * (X(0) @ Z(1) @ X(2)) + 0.25j * (X(0) @ Z(1) @ Y(2))
Pauli string of term 4: Adjoint(-0.25j * (Y(0) @ Z(1) @ X(2))) + Adjoint((0.25+0j) * (Y(0) @ Z(1) @ Y(2))) + Adjoint((0.25+0j) * (X(0) @ Z(1) @ X(2))) + Adjoint(0.25j * (X(0) @ Z(1) @ Y(2)))
Pauli string of term 5: (-0.125-0.12499999999999999j) * (Y(0) @ Z(1) @ Z(2) @ X(3)) + (0.12499999999999999-0.125j) * (Y(0) @ Z(1) @ Z(2) @ Y(3)) + (0.125+0.12499999999999999j) * (Y(0) @ Z(2) @ X(3)) + (-0.12499999999999999+0.125j) * (Y(0) @ Z(2) @ Y(3)) + (0.12499999999999999-0.125j) * (X(0) @ Z(1) @ Z(2) @ X(3)) + (0.125+0.12499999999999999j) * (X(0) @ Z(1) @ Z

In [17]:
import pennylane as qml
import numpy as np

# Set up a device (won’t run anything — just needed to define basis)
dev = qml.device("default.qubit", wires=4)

# Convert to matrix
mat = qml.matrix(H, wire_order=dev.wires)

# Diagonalize
eigvals_pennylane = np.linalg.eigvalsh(mat)
print("Eigenvalues from PennyLane JW Hamiltonian:")
print(np.round(np.sort(eigvals_pennylane), 6))


Eigenvalues from PennyLane JW Hamiltonian:
[-1.813607 -1.753143 -1.       -0.728185  0.        0.302729  0.318669
  0.470683  1.548682  2.333053  2.342923  2.357926  4.        4.296864
  5.323404  6.      ]


In [28]:
import pennylane as qml
from pennylane import numpy as np
import networkx as nx

# PARAMETERS
phi = np.pi
N = 4 # Number of sites
edges = [(1, 2), (2, 3), (2, 4), (1, 3), (3, 4)]
red_link = (2, 4) # Edge where Aharonov-Bohm phase applies
AB_phase = np.exp(1j * phi)

# GRAPH FOR PATHS
G = nx.Graph()
G.add_edges_from(edges)
nodes = sorted(set(i for e in edges for i in e))

def jw_path(i, j):
    path = nx.shortest_path(G, i, j)
    return path[1:-1] if len(path) > 2 else []

# PAPER-DEFINED OPERATORS

def SP(k):  # σ₊
    return 0.5 * (qml.PauliX(k) - 1j * qml.PauliY(k))

def SM(k):  # σ₋
    return 0.5 * (qml.PauliX(k) + 1j * qml.PauliY(k))

def number_op(k):  # n_x = 0.5 * (1 + Z)
    return 0.5 * (qml.Identity(k) + qml.PauliZ(k))

# def sigma_anyonic(k, phi):
#     alpha = np.exp(1j * phi)
#     return 0.5 * (alpha + 1) * qml.Identity(k) + 0.5 * (alpha - 1) * qml.PauliZ(k)

def sigma_anyonic(k, phi):
    if np.isclose(phi % (2*np.pi), np.pi):  # fermion case
        return number_op(k) * (-2)
    else:
        alpha = np.exp(1j * phi)
        return 0.5 * (alpha + 1) * qml.Identity(k) + 0.5 * (alpha - 1) * qml.PauliZ(k)

def anyon_string(path, phi):
    if not path:
        return qml.Identity(wires=range(N))
    result = sigma_anyonic(path[0], phi)
    for k in path[1:]:
        result = result @ sigma_anyonic(k, phi)
    return result

triples = [(i, j, jw_path(i, j)) for i in nodes for j in nodes if i < j]

ops = []
coeffs = []

for i, j, path in triples:
    i0, j0 = i - 1, j - 1
    path0 = [k - 1 for k in path]

    # Phase on red link only
    t = AB_phase if (i, j) == red_link or (j, i) == red_link else 1.0

    sigma_string = anyon_string(path0, phi)

    # a†_i σ a_j
    term1 = SP(i0) @ sigma_string @ SM(j0)
    term2 = term1.adjoint()

    ops.extend([term1, term2])
    coeffs.extend([t, np.conj(t)])

# ADD INTERACTION TERMS
V = 1.0
for i in range(N):
    for j in range(i + 1, N):
        ops.append(number_op(i) @ number_op(j))
        coeffs.append(V)

# BUILD HAMILTONIAN
H = qml.Hamiltonian(coeffs, ops)
Hmat = qml.matrix(H, wire_order=range(N))

# CHECK
print("Is H Hermitian?", np.allclose(Hmat, Hmat.conj().T))

# DIAGONALIZE
eigvals = np.linalg.eigvalsh(Hmat)
print("Eigenvalues from anyonic JW Hamiltonian:")
print(np.round(np.sort(eigvals), 6))


Is H Hermitian? True
Eigenvalues from anyonic JW Hamiltonian:
[-2.17741  -2.       -1.       -0.855773  0.        0.171573  0.321637
  1.        1.678363  2.        2.        2.855773  4.        4.17741
  5.828427  6.      ]


In [29]:
print("Pauli Strings in Hamiltonian:")
for i, op in enumerate(H.ops):
    # Convert the operator to its Pauli string representation
    pauli_string = str(op)
    # Print the Pauli string along with its coefficient
    print(f"Term {i + 1}: {pauli_string}    Coeff: {coeffs[i]}")


Pauli Strings in Hamiltonian:
Term 1: (0.5 * (X(0) + (-0-1j) * Y(0))) @ I([0, 1, 2, 3]) @ (0.5 * (X(1) + 1j * Y(1)))    Coeff: 1.0
Term 2: (Adjoint(0.5 * (X(1) + 1j * Y(1)))) @ (Adjoint(I([0, 1, 2, 3]))) @ (Adjoint(0.5 * (X(0) + (-0-1j) * Y(0))))    Coeff: 1.0
Term 3: (0.5 * (X(0) + (-0-1j) * Y(0))) @ I([0, 1, 2, 3]) @ (0.5 * (X(2) + 1j * Y(2)))    Coeff: 1.0
Term 4: (Adjoint(0.5 * (X(2) + 1j * Y(2)))) @ (Adjoint(I([0, 1, 2, 3]))) @ (Adjoint(0.5 * (X(0) + (-0-1j) * Y(0))))    Coeff: 1.0
Term 5: (0.5 * (X(0) + (-0-1j) * Y(0))) @ (-1.0 * (I(1) + Z(1))) @ (0.5 * (X(3) + 1j * Y(3)))    Coeff: 1.0
Term 6: (Adjoint(0.5 * (X(3) + 1j * Y(3)))) @ (Adjoint(-1.0 * (I(1) + Z(1)))) @ (Adjoint(0.5 * (X(0) + (-0-1j) * Y(0))))    Coeff: 1.0
Term 7: (0.5 * (X(1) + (-0-1j) * Y(1))) @ I([0, 1, 2, 3]) @ (0.5 * (X(2) + 1j * Y(2)))    Coeff: 1.0
Term 8: (Adjoint(0.5 * (X(2) + 1j * Y(2)))) @ (Adjoint(I([0, 1, 2, 3]))) @ (Adjoint(0.5 * (X(1) + (-0-1j) * Y(1))))    Coeff: 1.0
Term 9: (0.5 * (X(1) + (-0-1j) * Y